# Projeto 2 - Ciência dos Dados

Nome: Nevton de Castro

Nome: Omar Dibo

Nome: Laura Perim 

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [6]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [26]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle
import threading 

___
## Autenticando no  Twitter

* Conta: ***@Devsegundoseme1***

In [8]:
#Dados de autenticação do twitter: authpass

#Coloque aqui o identificador da conta no twitter: @Devsegundoseme1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:   
    data = json.load(fp)
    
#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'civic'

#Quantidade mínima de mensagens capturadas:
n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [9]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs1[:n])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs1[n:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()



NameError: name 'produto' is not defined

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [10]:
civic = pd.read_excel('civic.xlsx')
spm1 = civic[civic.valor==0] #tweets de spam ou venda
irr1 = civic[civic.valor==1] #tweets irrelevantes
rel1 = civic[civic.valor==2] #tweets de elogios ou criticas
irr = irr1['Treinamento'].tolist()
rel = rel1['Treinamento'].tolist()
spm = spm1['Treinamento'].tolist()

In [11]:
spm1.valor.value_counts() #40
rel1.valor.value_counts() #172
irr1.valor.value_counts() #538

1    538
Name: valor, dtype: int64

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:

nop = []
#nao vamos remover "#" nem "|"
for i in rel:
    punctuation = '[!\-.:?;,_+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop.append(text_subbed)


clean = []
for i in nop:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean.append(text_subbed)

relfinal = []
for i in clean:
    e = pd.Series(i.split())
    for i in e:
        relfinal.append(i)
df = pd.DataFrame(relfinal)
reldata = df[0].value_counts(True)
reldata.head(20)

o          0.035815
um         0.027767
de         0.023340
e          0.022938
que        0.022938
meu        0.018109
é          0.017706
honda      0.017706
do         0.013280
https      0.011670
//t        0.011670
eu         0.010463
a          0.009658
pra        0.008853
carro      0.008853
na         0.008853
mais       0.008451
não        0.008048
com        0.007646
corolla    0.007646
Name: 0, dtype: float64

In [13]:
nop2 = []
for i in irr:     
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop2.append(text_subbed)


clean2 = []
for i in nop2:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean2.append(text_subbed)


relfinal2 = []
for i in clean2:
    e = pd.Series(i.split())
    for i in e:
        relfinal2.append(i)
df2 = pd.DataFrame(relfinal2)
irrdata = df2[0].value_counts(True)
irrdata.head(20)

um       0.026008
de       0.024934
o        0.024099
e        0.021355
que      0.021236
a        0.018850
do       0.017060
honda    0.015867
//t      0.015867
https    0.015867
no       0.011095
eu       0.011095
meu      0.010021
na       0.009186
com      0.009186
é        0.008948
não      0.008590
mais     0.007277
carro    0.007158
pra      0.006442
Name: 0, dtype: float64

In [14]:

nop3 = []
#nao vamos remover "#" nem "|"
for i in spm:     
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop3.append(text_subbed)

clean3 = []
for i in nop3:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean3.append(text_subbed)

relfinal3 = []
for i in clean3:
    e = pd.Series(i.split())
    for i in e:
        relfinal3.append(i)
df3 = pd.DataFrame(relfinal3)
spmdata = df3[0].value_counts(True)
spmdata['https']

0.03492063492063492

In [15]:
l=['a','b','c','d','e','f']
t=['e','r','t','y','u','i']
for i in l, t:
    print(i)

['a', 'b', 'c', 'd', 'e', 'f']
['e', 'r', 't', 'y', 'u', 'i']


In [16]:

k = []
oi = civic['Treinamento'].tolist()

for i in oi:
    
    
    s = i.split()
    
    for t in s:
        
        k.append(i)

a = list(set(k))
td = len(a)

k=[]
for i in clean:
    s = i.split()
    for t in s:
        k.append(i)
        
tdrel = len(k)

k=[]
for i in clean2:
    s = i.split()
    for t in s:
        k.append(i)
        
tdirr = len(k)

k=[]
for i in clean3:
    s = i.split()
    for t in s:
        k.append(i)
        
tdspm = len(k)
print(tdrel)
print(tdirr)
print(tdspm)

2485
8382
1260


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [17]:
lista =[]
d=-1
for i in clean:
    t=1
    
    e = pd.Series(i.split())
    
    for w in e:
        
        if w in df[0].tolist():
            r=reldata[w]
        else:
            r=0
        t = r*t
    t*(172/750)
    o=[]
    o.append(t)
    lista.append(o)
    
d = -1
for i in clean:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df2[0].tolist():
            r = irrdata[w]
        else:
            r=0
        t = r*t
    t*(538/750)
    lista[d].append(t)
    
d = -1
for i in clean:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df3[0].tolist():
            r = spmdata[w]
        else:
            r=0
        t = r*t
    t*(40/750)
    lista[d].append(t)
qual=[]
m=-1
for i in lista:
    m=m+1
    if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]):
        qual.append('relevante')
    if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]):
        qual.append('irrelevante')
    if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]):
        qual.append('spam')


In [18]:
lista =[]

d=-1
for i in clean2:
    t=1
    
    e = pd.Series(i.split())
    
    for w in e:
        
        if w in df[0].tolist():
            r=reldata[w]
        else:
            r=0
        t = r*t
    t*(172/750)
    o=[]
    o.append(t)
    lista.append(o)


d = -1
for i in clean2:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df2[0].tolist():
            r = irrdata[w]
        else:
            r=0
        t = r*t
    t*(538/750)
    lista[d].append(t)
    
d = -1
for i in clean2:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df3[0].tolist():
            r = spmdata[w]
        else:
            r=0
        t = r*t
    t*(40/750)
    lista[d].append(t)
qual=[]
m=-1
for i in lista:
    m=m+1
    if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]):
        qual.append('relevante')
    if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]):
        qual.append('irrelevante')
    if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]):
        qual.append('spam')
    


In [19]:
lista =[]

d=-1
for i in clean3:
    t=1
    
    e = pd.Series(i.split())
    
    for w in e:
        
        if w in df[0].tolist():
            r=reldata[w]
        else:
            r=0
        t = r*t
    t*(172/750)
    o=[]
    o.append(t)
    lista.append(o)


d = -1
for i in clean3:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df2[0].tolist():
            r = irrdata[w]
        else:
            r=0
        t = r*t
    t*(538/750)
    lista[d].append(t)
    
d = -1
for i in clean3:
    t =1
    d =d+1
    e = pd.Series(i.split())
    for w in e:
        if w in df3[0].tolist():
            r = spmdata[w]
        else:
            r=0
        t = r*t
    t*(40/750)
    lista[d].append(t)
qual=[]
m=-1
for i in lista:
    m=m+1
    if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]):
        qual.append('relevante')
    if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]):
        qual.append('irrelevante')
    if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]):
        qual.append('spam')
    


In [20]:
um = df[0].value_counts()
dois = df2[0].value_counts()
tres = df3[0].value_counts()

In [21]:
def p(tweets):
    lista =[]
    
    d=-1
    for i in tweets:
        t=1

        e = pd.Series(i.split())
        
        for w in e:
            if w in df[0].tolist():
                r = (df[0].value_counts()[w]+1)/(tdrel + td)
            else:
                r = 1/(tdrel + td)
            t = r*t
        t*(172/750)
        o=[]
        o.append(t)
        lista.append(o)


    d = -1
    for i in tweets:
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:
            if w in df2[0].tolist():
                r = (df2[0].value_counts()[w]+1)/(tdirr + td)
            else:
                r= 1/(tdirr + td)
            t = r*t
        t*(538/750)
        lista[d].append(t)

    d = -1
    for i in tweets:
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:
            if w in df3[0].tolist():
                r = (df3[0].value_counts()[w]+1)/(tdspm + td)
            else:
                r= 1/(tdspm+td)
            t = r*t
        t*(40/750)
        lista[d].append(t)
    qual=[]
    m=-1
    for i in lista:
        m=m+1
        if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]):
            qual.append('relevante')
        if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]):
            qual.append('irrelevante')
        if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]):
            qual.append('spam')
    return qual



In [22]:
g=0
b=0
erros = 0
for i in p(clean):
    if i != 'relevante':
        erros = erros+1

erros

6

In [23]:

for i in p(clean2):
    if i != 'irrelevante':
        erros = erros+1


erros

287

In [25]:
for i in p(clean3):
    if i != 'spam':
        erros = erros+1
erros

287

In [27]:

  
timer = threading.Timer(300.0, p()) 
timer.start() 
print("Acuracia") 

TypeError: p() missing 1 required positional argument: 'tweets'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**